In [ ]:
import numpy as np
import os
from PIL import Image
from tensorflow.keras.models import load_model
from tensorflow.keras.losses import BinaryCrossentropy, MeanSquaredError

def combined_loss(y_true, y_pred):
    bce = BinaryCrossentropy()(y_true, y_pred)
    mse = MeanSquaredError()(y_true, y_pred)
    return bce + 0.01 * mse

def load_images(image_folder, target_size=(256, 256)):
    images = []
    image_files = [f for f in sorted(os.listdir(image_folder)) if f.endswith(('.jpg', '.jpeg', '.png'))]
    for image_file in image_files:
        image_path = os.path.join(image_folder, image_file)
        image = Image.open(image_path).convert('L')
        image = image.resize(target_size)
        image = np.array(image) / 255.0  # Normalize pixel values to [0, 1]
        images.append(image)
    return np.array(images), image_files

def run_inference(model, images):
    images = np.expand_dims(images, axis=-1)
    predictions = model.predict(images)
    return predictions

def save_predictions(predictions, image_files, save_dir):
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)
    
    for i, prediction in enumerate(predictions):
        prediction = (prediction * 255).astype(np.uint8)
        pred_image = Image.fromarray(prediction.squeeze(), mode='L')
        save_path = os.path.join(save_dir, image_files[i])
        pred_image.save(save_path)
        print(f"Saved prediction to {save_path}")

# Load the model
model_save_path = 'path/to/your/model.h5'
model = load_model(model_save_path, custom_objects={'combined_loss': combined_loss})

print("Model loaded from disk")

# Directories
image_folder = 'path/to/your/input_images'
save_dir = 'path/to/your/predictions'

# Load and preprocess images
images, image_files = load_images(image_folder, target_size=(256, 256))

# Run inference
predictions = run_inference(model, images)

# Save predictions
save_predictions(predictions, image_files, save_dir)
